# RAG application built on gemini 

In [5]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("yolov9_paper.pdf")
data = loader.load()  # entire PDF is loaded as a single Document
#data

In [3]:
len(data)

18

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# split data
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data)


print("Total number of documents: ",len(docs))

Total number of documents:  96


In [8]:
docs[7]

Document(metadata={'source': 'yolov9_paper.pdf', 'page': 1}, page_content='ditional layers to combine repeatedly fed input data, which\nwill significantly increase the inference cost. In addition,\nsince the input data layer to the output layer cannot have a\ntoo deep path, this limitation will make it difficult to model\nhigh-order semantic information during the training pro-\ncess. As for masked modeling, its reconstruction loss some-\ntimes conflicts with the target loss. In addition, most mask\nmechanisms also produce incorrect associations with data.\nFor the deep supervision mechanism, it will produce error\naccumulation, and if the shallow supervision loses informa-\ntion during the training process, the subsequent layers will\nnot be able to retrieve the required information. The above\nphenomenon will be more significant on difficult tasks and\nsmall models.\nTo address the above-mentioned issues, we propose a\nnew concept, which is programmable gradient information\n(PGI). T

In [9]:
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

from dotenv import load_dotenv
load_dotenv() 

#Get an API key: 
# Head to https://ai.google.dev/gemini-api/docs/api-key to generate a Google AI API key. Paste in .env file

# Embedding models: https://python.langchain.com/v0.1/docs/integrations/text_embedding/

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector = embeddings.embed_query("hello, world!")
vector[:5]
#vector

[0.05168594419956207,
 -0.030764883384108543,
 -0.03062233328819275,
 -0.02802734449505806,
 0.01813092641532421]

In [10]:
vectorstore = Chroma.from_documents(documents=docs, embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"))

In [11]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})

retrieved_docs = retriever.invoke("What is new in yolov9?")


In [13]:
len(retrieved_docs)

10

In [17]:
print(retrieved_docs[5].page_content)

YOLOv9: Learning What You Want to Learn
Using Programmable Gradient Information
Chien-Yao Wang1,2, I-Hau Yeh2, and Hong-Yuan Mark Liao1,2,3
1Institute of Information Science, Academia Sinica, Taiwan
2National Taipei University of Technology, Taiwan
3Department of Information and Computer Engineering, Chung Yuan Christian University, Taiwan
kinyiu@iis.sinica.edu.tw, ihyeh@emc.com.tw, and liao@iis.sinica.edu.tw
Abstract
Today’s deep learning methods focus on how to design
the most appropriate objective functions so that the pre-
diction results of the model can be closest to the ground
truth. Meanwhile, an appropriate architecture that can
facilitate acquisition of enough information for prediction
has to be designed. Existing methods ignore a fact that
when input data undergoes layer-by-layer feature extrac-
tion and spatial transformation, large amount of informa-
tion will be lost. This paper will delve into the important is-


In [18]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro",temperature=0.3, max_tokens=500)

In [19]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [20]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [21]:
response = rag_chain.invoke({"input": "what is new in YOLOv9?"})
print(response["answer"])

YOLOv9 introduces a new architecture called GELAN, which improves parameter usage and inference speed. It also proposes a novel Programmable Gradient Information (PGI) mechanism for better gradient flow during training. These improvements lead to state-of-the-art performance on object detection tasks, surpassing previous YOLO versions and other real-time detectors. 

